## Chronos
El modelo Chronos se diferencia al entrenar desde cero en una amplia colección de series temporales, empleando una tokenización que convierte valores continuos en una secuencia de tokens discretos. A diferencia de otros métodos que necesitan ajustes dentro del dominio, Chronos utiliza una distribución categórica para modelar observaciones, lo que le permite realizar regresiones a través de la clasificación sin características específicas para series temporales. Este enfoque minimalista no solo mejora la capacidad de predicción en escenarios de zero-shot, donde el modelo predice sin haber visto previamente los datos, sino que también sugiere un potencial para tareas adicionales como la imputación de datos, la clasificación y la detección de anomalías en series temporales. 

Chronos utiliza una función de pérdida basada en categorical cross entropy, similar a los modelos de lenguaje tradicionales, pero aplicada a la predicción de series temporales tokenizadas. La pérdida mide la diferencia entre la distribución de etiquetas cuantizadas reales y la distribución predicha por el modelo. Un punto clave es que la función de pérdida no es consciente de las distancias entre bins adyacentes, por lo que el modelo debe aprender a asociar bins cercanos basándose en la distribución de los índices de bins en los datos de entrenamiento. 

Chronos es un modelo probabilístico que genera múltiples realizaciones del futuro mediante un muestreo autoregresivo de la distribución predicha. Las trayectorias de muestra se presentan en forma de IDs de tokens que luego deben ser mapeados de nuevo a valores reales y desescalados para obtener la previsión real. La función de desescalado invierte la transformación aplicada durante el escalado inicial, devolviendo los valores a su escala original, facilitando así la obtención de predicciones precisas a partir de los tokens generados por el modelo. 

##### Análisis de Hiperparámetros 
El estudio evaluó varios aspectos para mejorar el rendimiento de los modelos de pronóstico de series temporales, como el tamaño del modelo, la inicialización, el uso de aumentaciones TSMix, la proporción de datos sintéticos, los pasos de entrenamiento, la longitud del contexto y el tamaño del vocabulario. Los resultados mostraron que aumentar el tamaño del modelo mejora el rendimiento, pero el uso de pesos preentrenados de modelos de lenguaje no fue efectivo, siendo la inicialización aleatoria superior en este caso. Las aumentaciones TSMix y pequeñas cantidades de datos sintéticos mejoraron el rendimiento en escenarios cero-shot, pero usar demasiados datos sintéticos afectó negativamente debido a la falta de representatividad de los datos reales. Más pasos de entrenamiento y una mayor longitud de contexto mejoraron el rendimiento, especialmente en datos de alta frecuencia. El tamaño del vocabulario también influyó en algunas métricas, lo que resalta la importancia de seleccionar las métricas adecuadas para la evaluación de modelos.d

In [0]:
# !pip install git+https://github.com/amazon-science/chronos-forecasting.git

In [0]:
%pip install git+https://github.com/amazon-science/chronos-forecasting.git
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Cloning https://github.com/amazon-science/chronos-forecasting.git to /tmp/pip-req-build-p3ahq26t
  Running command git clone --filter=blob:none --quiet https://github.com/amazon-science/chronos-forecasting.git /tmp/pip-req-build-p3ahq26t
  Resolved https://github.com/amazon-science/chronos-forecasting.git to commit ac6ee36acee1e47446cd66f72f540c87f1f1fbe4
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, DoubleType, StructType, StructField, FloatType


In [0]:
from databricks.sdk.runtime import spark

In [0]:

from src.models.train_test_split import create_train_test_df
from src.models.chronos_functions import create_get_horizon_timestamps, create_forecast_udf, generate_forecast_with_comparison
from src.metrics.models_metrics import calculate_metrics

2024-10-07 18:38:34.716475: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-07 18:38:34.722180: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-07 18:38:34.792415: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-07 18:38:36.274263: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [0]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())

False
0


## Load Data


In [0]:
# Read the CSV file
csv_df = pd.read_csv('dataset/dataset_01.csv')

# CSV file has two columns: 'unique_id' for the time series identifier and 'y' for the values
# If you have a 'date' column, you can rename it to 'ds' for compatibility
csv_df = csv_df.rename(columns={"TRAN_START_DT": "ds", "QUANTITY": "y"})


In [0]:
# Prepare DataFrame
csv_df['unique_id'] = csv_df['LOCATION_ID'].astype(str) + '_' + csv_df['ITEM_ID'].astype(str) 
csv_df['ds'] = pd.to_datetime(csv_df['ds'])
csv_df = csv_df.drop(columns=['LOCATION_ID', 'ITEM_ID'])
csv_df.head()

,ds,y,unique_id
0,2022-01-02,98.0,J519_1556
1,2022-01-03,86.0,J519_1556
2,2022-01-04,77.0,J519_1556
3,2022-01-05,73.0,J519_1556
4,2022-01-06,67.0,J519_1556


In [0]:
csv_df.unique_id.unique()

array(['J519_1556', 'J519_113746', 'J519_113752', ..., 'J519_1855213002',
       'J519_1858664001', 'J519_1883951002'], dtype=object)

In [0]:
# csv_df = csv_df.loc[csv_df.unique_id.isin(['J519_1556', 'J519_1858664001', 'J519_113752', 'J519_1883951002'])]
csv_df = csv_df.loc[csv_df.unique_id.isin(csv_df.unique_id.unique()[:500])]

2. Adjust Inference and Forecasting
Once the data is loaded and prepared, the rest of the script should remain the same. You can apply the UDFs to generate forecasts based on your data:

- get_horizon_timestamps: This function generates a Pandas UDF that creates future timestamps for a given time series. The UDF is responsible for adding the required timestamps based on the frequency (freq) and the number of predictions (prediction_length)

- forecast_udf: The forecast_udf is another Pandas UDF that generates the forecast using a pretrained model from Chronos (loaded from amazon/{chronos_model}). It predicts prediction_length future values for each time series, based on the historical data.

## Forecasting Chronos

In [0]:
# chronos_type = "tiny"
chronos_type = "small"

In [0]:
train_df, test_df = create_train_test_df(csv_df, prediction_length=30)
# test_df.show()

In [0]:
chronos_froecast = generate_forecast_with_comparison(train_df, 
                                                     chronos_model=f"chronos-t5-{chronos_type}", 
                                                     prediction_length=30, 
                                                     num_samples=10, 
                                                     batch_size=30, 
                                                     freq="D", 
                                                     device_map='cpu')

In [0]:
# chronos_froecast.show()

### Metrics

**MAPE (Mean Absolute Percentage Error)**
- **Definición**: Mide el error promedio en términos porcentuales entre los valores reales y los valores predichos por un modelo.
- **Fórmula**:
  $$
  \text{MAPE} = \frac{1}{n} \sum_{i=1}^{n} \left| \frac{y_i - \hat{y_i}}{y_i} \right| \times 100
  $$
  Donde:
  - yi: Valor real.
  - ŷi: Valor predicho.
  - n: Número de observaciones.



**MASE (Mean Absolute Scaled Error)**

- **Definición**: Compara el error absoluto promedio de un modelo de pronóstico con el error de un pronóstico de referencia. Está escalado, lo que permite comparaciones entre diferentes series temporales o modelos.
- **Fórmula**:
  $$
  \text{MASE} = \frac{\frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y_i}|}{\frac{1}{n-1} \sum_{i=2}^{n} |y_i - y_{i-1}|}
  $$
  Donde:
  - El denominador es el error promedio de un pronóstico naive que usa la diferencia entre los valores reales y su valor anterior.



**R² (Coeficiente de Determinación)**

- **Definición**: Mide qué tan bien se ajusta el modelo a los datos reales. Representa la proporción de la varianza en la variable dependiente que es explicada por las variables independientes.
- **Fórmula**:
  $$
  R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y_i})^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}
  $$
  Donde:
  - yi: Valores reales.
  - ŷi: Valores predichos.
  - ȳ: Promedio de los valores reales.




In [0]:
metrics_schema = StructType([
    StructField('unique_id', StringType(), True), 
    StructField('r2', DoubleType(), True), 
    StructField('mape', DoubleType(), True), 
    StructField('mase', DoubleType(), True)
    ])

In [0]:
joined_df = chronos_froecast.join(test_df, on="unique_id")

# Select only the required columns
joined_df = joined_df.select("unique_id", "forecast", "test_y")
# joined_df.show()

In [0]:
# Use applyInPandas to calculate R² and MAPE for each unique_id
joined_df.cache()
metrics_df = joined_df.groupBy("unique_id").applyInPandas(calculate_metrics, metrics_schema)

In [0]:
# metrics_df.show()

In [0]:
metrics_df.createOrReplaceTempView("df_forecast_chronos_metrics_temp")

In [0]:
spark.sql(f"CREATE OR REPLACE TABLE dev.df_forecastu_chronos_{chronos_type}_metrics AS SELECT * FROM df_forecast_chronos_metrics_temp")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
select *
from dev.df_forecastu_chronos_small_metrics

unique_id,r2,mape,mase
J519_120660,-0.34566037735849053,63.05925925925925,0.7562745098039216
J519_1222065,-0.435063381966037,40.79365079365079,0.7279835390946502
J519_1729699,-0.6385231825987405,70.39550264550265,0.7558510638297873
J519_1801353,-0.5918267945022015,31.605758357452945,1.138924924924925
J519_1856076,-0.17206601466992666,1.1000000000549739E12,1.112673611111111
J519_1885273,-0.15261958997722092,5.500000000478612E11,0.8752252252252252
J519_1896346,-0.10746129204484767,72.36150396444515,0.8761375661375661
J519_1899350,0.16520485559359788,63.85132154172922,0.8682941176470589
J519_269410,-0.46916292495189227,48.781746031746025,0.8738157894736842
J519_1051189,0.6096390367516948,24.876020154254395,0.6005443698732288


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

-  MASE mínimo de 0.5 aprox. significa que, al menos en algunas de las series, el modelo tiene la mitad del error del modelo ingenuo (Nive model), lo que sugiere un rendimiento bastante bueno para esas series
- MASE de 0.8, significa que el modelo generalmente está haciendo un trabajo mejor que el modelo ingenuo (ya que es menor a 1), pero el margen de mejora es pequeño. El modelo es solo un 20% menor que ele error del modelo ingenuo, captura patrones pero no de manera muy eficiente.
- MASE mayor a 1, infica que está comentiendo mas errores que el modelo ingenuo, no esta2 captruando bien las tendencias y patrones.

Según el paper [TImeGPT-1](https://arxiv.org/html/2310.03589v2#:~:text=In%20summary%2C%20we%20observed%20that,significantly%20drops%20in%20inference%20speed.), se observa que TimeGPT supera de manera consistente a los modelos básicos recientes con un tiempo de inferencia comparable. A continuación, TimesFM ocupa el segundo lugar en términos de precisión con un tiempo de inferencia similar al de TimeGPT. Amazon Chronos ocupa el tercer lugar en precisión, pero cae significativamente en velocidad de inferencia. Por último, si bien Moirari y LagLlama son más eficientes en velocidad de inferencia que Chronos, ocupan un lugar más bajo en precisión por un amplio margen que todos los demás modelos.

Recursos:
- https://github.com/amazon-science/chronos-forecasting/tree/main?tab=readme-ov-file
- https://arxiv.org/html/2310.03589v2#:~:text=In%20summary%2C%20we%20observed%20that,significantly%20drops%20in%20inference%20speed.